In [20]:
import pandas as pd
from datetime import datetime
import random

# Define the date range
start_date = datetime(2022, 8, 1)
end_date = datetime.now()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Define income, rent, and other fixed costs
income_amount = 1500.00  # Fixed monthly income
rent_amount = 500.00  # Fixed rent amount

# Define utility bills
utility_bills = {
    'Electricity Bill': (60, 150),
    'Water Bill': (30, 60),
    'Internet Bill': (45, 45)  # Fixed Internet Bill
}

# Define streaming service cost choices
streaming_options = [7, 10, 16, 20, 30]

# Set monthly cap and initialize carryover savings
monthly_cap = income_amount * 0.8  # Limit for expenses to ensure savings
carryover_savings = 0

# Create an empty list to store data
data = []

# Track current month for monthly budget
current_month = None
streaming_subscription = 0  # Initial streaming subscription cost
for date in date_range:
    # Determine if it's a holiday or summer break month
    is_december = date.month == 12
    is_summer = date.month in [6, 7]

    # Monthly setup
    if current_month != date.month:
        current_month = date.month
        current_year = date.year
        monthly_expenses = 0
        monthly_food_count = 0
        grocery_count = 0
        shopping_count = 0
        streaming_expense = 0
        movie_ticket_added = False
        streaming_subscription = random.choice(streaming_options) if date.month != 8 or date.year != 2022 else 0
        no_transaction_days = random.sample(range(1, 29), 8 if is_summer or is_december else 4)

    # Income on the 1st of every month
    if date.day == 1:
        data.append([date.strftime('%Y-%m-%d'), 'Income', 'Income', income_amount])
        # Adjust monthly budget by adding carryover savings
        monthly_budget = monthly_cap + carryover_savings

    # Gym membership on the 1st (may reduce in summer)
    if date.day == 1 and not is_summer:
        gym_fee = 25.00
        if monthly_expenses + gym_fee <= monthly_budget:
            data.append([date.strftime('%Y-%m-%d'), 'Gym', 'Gym Membership', gym_fee])
            monthly_expenses += gym_fee

    # Rent and Utilities on the 5th of each month
    if date.day == 5:
        if date.month == 8 and date.year == 2022:
            # Only rent for August 2022
            if monthly_expenses + rent_amount <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Utilities', 'Rent', rent_amount])
                monthly_expenses += rent_amount
        else:
            # Full utilities from September 2022 onward
            electricity_bill = round(random.uniform(60, 150), 2)
            water_bill = round(random.uniform(30, 60), 2)
            internet_bill = 45.00
            utilities_total = rent_amount + electricity_bill + water_bill + internet_bill
            if monthly_expenses + utilities_total <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Utilities', 'Rent', rent_amount])
                data.append([date.strftime('%Y-%m-%d'), 'Utilities', 'Electricity Bill', electricity_bill])
                data.append([date.strftime('%Y-%m-%d'), 'Utilities', 'Water Bill', water_bill])
                data.append([date.strftime('%Y-%m-%d'), 'Utilities', 'Internet Bill', internet_bill])
                monthly_expenses += utilities_total

    # Streaming Service: Use current subscription if under limit, or pick new
    if streaming_expense + streaming_subscription <= 30 and streaming_subscription:
        if random.choice([True, False]):
            data.append([date.strftime('%Y-%m-%d'), 'Entertainment', 'Streaming Service', streaming_subscription])
            monthly_expenses += streaming_subscription
            streaming_expense += streaming_subscription
        else:
            streaming_subscription = random.choice(streaming_options)

    # Weekday Spending during Semester: Limited (taxi for essentials)
    if not is_summer and not is_december and date.weekday() < 5:
        if grocery_count < 3:
            grocery_expense = round(random.uniform(50, 150), 2)
            if monthly_expenses + grocery_expense <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Food', 'Groceries', grocery_expense])
                monthly_expenses += grocery_expense
                grocery_count += 1
                monthly_food_count += 1
        elif monthly_food_count < 10 and random.choice([True, False]):
            dine_out_expense = round(random.uniform(50, 150), 2)
            if monthly_expenses + dine_out_expense <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Food', 'Dine-out', dine_out_expense])
                monthly_expenses += dine_out_expense
                monthly_food_count += 1

        # Transportation: Taxi on random weekdays for essentials
        if random.choice([True, False, False]) and monthly_expenses + 20 <= monthly_budget:
            taxi_expense = round(random.uniform(5, 20), 2)
            data.append([date.strftime('%Y-%m-%d'), 'Transportation', 'Taxi', taxi_expense])
            monthly_expenses += taxi_expense

    # Weekend or Summer Break Spending: Groceries, dine-outs, entertainment more likely
    elif is_summer or date.weekday() >= 5 or is_december:
        if grocery_count < 3 or random.choice([True, False]):
            grocery_expense = round(random.uniform(50, 150), 2)
            if monthly_expenses + grocery_expense <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Food', 'Groceries', grocery_expense])
                monthly_expenses += grocery_expense
                grocery_count += 1
                monthly_food_count += 1
        if monthly_food_count < 10 and random.choice([True, False]):
            dine_out_expense = round(random.uniform(50, 150), 2)
            if monthly_expenses + dine_out_expense <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Food', 'Dine-out', dine_out_expense])
                monthly_expenses += dine_out_expense
                monthly_food_count += 1

        # Movie Ticket on weekends, especially in summer and holidays
        if (date.day == 10 or random.choice([True, False, False])) and not movie_ticket_added:
            if monthly_expenses + 20 <= monthly_budget:
                data.append([date.strftime('%Y-%m-%d'), 'Entertainment', 'Movie Ticket', 20.00])
                monthly_expenses += 20.00
                movie_ticket_added = True

    # Holiday or Summer Break: Reduced or Special Expenses
    if is_december and random.choice([True, False, False]):
        holiday_shopping = round(random.uniform(100, 300), 2)
        if monthly_expenses + holiday_shopping <= monthly_budget:
            data.append([date.strftime('%Y-%m-%d'), 'Entertainment', 'Holiday Shopping', holiday_shopping])
            monthly_expenses += holiday_shopping

    # "No Transactions" days for selected days
    if date.day in no_transaction_days:
        data.append([date.strftime('%Y-%m-%d'), 'No Transactions', 'No Transactions', 0.00])

    # End of month: calculate carryover savings
    if date == date_range[-1] or (date + pd.Timedelta(days=1)).month != current_month:
        carryover_savings = income_amount - monthly_expenses

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=['Date', 'Category', 'Description', 'Amount'])

# Save to CSV
df.to_csv('income_and_utilities_expense_tracker.csv', index=False)

print("CSV file created: income_and_utilities_expense_tracker.csv")

CSV file created: income_and_utilities_expense_tracker.csv
